# Project 2 - EDA on Vehicle Insurance Customer Data

In [1]:
import pandas as pd
import numpy as np

### 1. Add the column names to both datasets:

In [39]:
## load customer_details.csv file as cdt (cdt= customer details table)
cdt = pd.read_csv("customer_details.csv",header=0)

In [40]:
## set columns for cdt
cdt.rename({'0':'Customer_Id','1':'Gender','2':'Age','3':'Driving Licence Present','4':'Region Code','5':'Previously Insured','6':'Vehicle Age','7':'Vehicle Damage'},axis=1,inplace=True)

In [41]:
cdt.head()

,Customer_Id,Gender,Age,Driving Licence Present,Region Code,Previously Insured,Vehicle Age,Vehicle Damage
0,1.0,Male,44.0,1.0,28.0,0.0,> 2 Years,Yes
1,2.0,Male,76.0,1.0,3.0,0.0,1-2 Year,No
2,3.0,Male,47.0,1.0,28.0,0.0,> 2 Years,Yes
3,4.0,Male,21.0,1.0,11.0,1.0,< 1 Year,No
4,5.0,Female,29.0,1.0,41.0,1.0,< 1 Year,No


In [42]:
## load customer_policy_details.csv as cpt (cpt = customer policy table)
cpt = pd.read_csv('customer_policy_details.csv')

In [43]:
## set columns for cpt
cpt.rename({'0':'Customer_Id','1':'Annual Premium(Rs)','2':'Sales Channel Code','3':'Vintage','4':'Response'},axis=1,inplace=True)

In [44]:
cpt.head()

,Customer_Id,Annual Premium(Rs),Sales Channel Code,Vintage,Response
0,1.0,40454.0,26.0,217.0,1.0
1,2.0,33536.0,26.0,183.0,0.0
2,3.0,38294.0,26.0,27.0,1.0
3,4.0,28619.0,152.0,203.0,0.0
4,5.0,27496.0,152.0,39.0,0.0


### 2. Checking and Cleaning Data Quality:

#### i. Null values

    -> Generate a summary of count of all the null values column wise
    -> Drop Null values for customer_id because central tendencies for id’s is not feasible.
    -> Replace all null values for numeric columns by mean. 
    -> Replace all null values for Categorical value by mode.

In [45]:
## for table cdt/ csv file customer_details.csv 
cdt.isnull().sum() ## all null values column wise

Customer_Id                386
Gender                     368
Age                        368
Driving Licence Present    393
Region Code                392
Previously Insured         381
Vehicle Age                381
Vehicle Damage             407
dtype: int64

In [46]:
cdt = cdt.dropna(subset=['Customer_Id'])
cdt.isna().sum()

Customer_Id                  0
Gender                     368
Age                        366
Driving Licence Present    392
Region Code                391
Previously Insured         381
Vehicle Age                381
Vehicle Damage             406
dtype: int64

In [47]:
## Replace all null values for numeric columns by mean
cdt = cdt.fillna(cdt.mean(numeric_only=True))
cdt.isnull().sum()

Customer_Id                  0
Gender                     368
Age                          0
Driving Licence Present      0
Region Code                  0
Previously Insured           0
Vehicle Age                381
Vehicle Damage             406
dtype: int64

In [48]:
## Replace all null values for numeric columns by mean
cdt['Gender']=cdt['Gender'].fillna(cdt['Gender'].mode()[0])
cdt['Vehicle Age']=cdt['Vehicle Age'].fillna(cdt['Vehicle Age'].mode()[0])
cdt['Vehicle Damage']=cdt['Vehicle Damage'].fillna(cdt['Vehicle Damage'].mode()[0])
cdt.isnull().sum()

Customer_Id                0
Gender                     0
Age                        0
Driving Licence Present    0
Region Code                0
Previously Insured         0
Vehicle Age                0
Vehicle Damage             0
dtype: int64

In [49]:
## for table cpt/ csv file customer_policy_details.csv
cpt.isnull().sum() ## all null values column wise

Customer_Id           387
Annual Premium(Rs)    346
Sales Channel Code    400
Vintage               388
Response              361
dtype: int64

In [50]:
cpt = cpt.dropna(subset=['Customer_Id'])
cpt.isna().sum()

Customer_Id             0
Annual Premium(Rs)    344
Sales Channel Code    400
Vintage               388
Response              361
dtype: int64

In [51]:
## Replace all null values for numeric columns by mean
cpt = cpt.fillna(cpt.mean(numeric_only=True))
cpt.isnull().sum()

Customer_Id           0
Annual Premium(Rs)    0
Sales Channel Code    0
Vintage               0
Response              0
dtype: int64

#### ii. Outliers

    -> Generate a summary of count of all the outliers column wise
    -> Replace all outlier values for numeric columns by mean. 
    (Hint1: for outlier treatment use IQR method as follows:

    For example: for a column X calculate Q1 = 25th percentile and Q3 = 75th percentile then IQR = Q3 – Q1 ) then to check outlier, anything lower than a Q1 – 1.5IQR or greater than Q3 + 1.5 IQR would be an outlier

    Hint2: For getting percentile value, explore pd.describe() function)

In [52]:
## count of all the outliers column wise for customer policy table (cpt)
Q1=cpt.quantile(0.25)
Q3=cpt.quantile(0.75)
IQR=Q3-Q1
l=Q1-(1.5*IQR)## for lower values from Q1
u=Q3+(1.5*IQR)## for upper/greater values from Q3
((cpt<l)|(cpt>u)).sum()

Customer_Id               0
Annual Premium(Rs)    10332
Sales Channel Code        0
Vintage                   0
Response              46965
dtype: int64

In [53]:
cpt['Annual Premium(Rs)'].describe()

count    380722.000000
mean      30563.999774
std       17190.147550
min        2630.000000
25%       24416.000000
50%       31656.000000
75%       39391.750000
max      540165.000000
Name: Annual Premium(Rs), dtype: float64

In [54]:
### Replace all outlier values for numeric columns by mean. 

q1=cpt['Annual Premium(Rs)'].quantile(0.25)
q3=cpt['Annual Premium(Rs)'].quantile(0.75)

IQR=q3-q1

upper = cpt['Annual Premium(Rs)'][~(cpt['Annual Premium(Rs)']>(q1+1.5*IQR))].max()

lower = cpt['Annual Premium(Rs)'][~(cpt['Annual Premium(Rs)']<(q1-1.5*IQR))].min()

In [57]:
cpt['Annual Premium(Rs)'] = np.where(cpt['Annual Premium(Rs)'] > upper,cpt['Annual Premium(Rs)'].mean(),np.where(cpt['Annual Premium(Rs)'] < lower,cpt['Annual Premium(Rs)'].mean(),cpt['Annual Premium(Rs)']))

#### iii. White spaces
    Remove white spaces

In [62]:
## for customer details tabe 
cdt['Gender'].str.strip()
cdt['Vehicle Age'].str.strip()
cdt['Vehicle Damage'].str.strip()

0         Yes
1          No
2         Yes
3          No
4          No
         ... 
381104     No
381105     No
381106     No
381107    Yes
381108     No
Name: Vehicle Damage, Length: 380723, dtype: object

#### iv. case correction(lower or upper, any one)

In [63]:
## for customer details tabe
cdt.apply(lambda x: x.astype(str).str.upper())

,Customer_Id,Gender,Age,Driving Licence Present,Region Code,Previously Insured,Vehicle Age,Vehicle Damage
0,1.0,MALE,44.0,1.0,28.0,0.0,> 2 YEARS,YES
1,2.0,MALE,76.0,1.0,3.0,0.0,1-2 YEAR,NO
2,3.0,MALE,47.0,1.0,28.0,0.0,> 2 YEARS,YES
3,4.0,MALE,21.0,1.0,11.0,1.0,< 1 YEAR,NO
4,5.0,FEMALE,29.0,1.0,41.0,1.0,< 1 YEAR,NO
...,...,...,...,...,...,...,...,...
381104,381105.0,MALE,74.0,1.0,26.0,1.0,1-2 YEAR,NO
381105,381106.0,MALE,30.0,1.0,37.0,1.0,< 1 YEAR,NO
381106,381107.0,MALE,21.0,1.0,30.0,1.0,< 1 YEAR,NO
381107,381108.0,FEMALE,68.0,1.0,14.0,0.0,> 2 YEARS,YES


#### v. Convert nominal data (categorical) into dummies 

In [65]:
## for customer details table 
customer_details=pd.get_dummies(cdt)

In [66]:
customer_details

,Customer_Id,Age,Driving Licence Present,Region Code,Previously Insured,Gender_Female,Gender_Male,Vehicle Age_1-2 Year,Vehicle Age_< 1 Year,Vehicle Age_> 2 Years,Vehicle Damage_No,Vehicle Damage_Yes
0,1.0,44.0,1.0,28.0,0.0,0,1,0,0,1,0,1
1,2.0,76.0,1.0,3.0,0.0,0,1,1,0,0,1,0
2,3.0,47.0,1.0,28.0,0.0,0,1,0,0,1,0,1
3,4.0,21.0,1.0,11.0,1.0,0,1,0,1,0,1,0
4,5.0,29.0,1.0,41.0,1.0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
381104,381105.0,74.0,1.0,26.0,1.0,0,1,1,0,0,1,0
381105,381106.0,30.0,1.0,37.0,1.0,0,1,0,1,0,1,0
381106,381107.0,21.0,1.0,30.0,1.0,0,1,0,1,0,1,0
381107,381108.0,68.0,1.0,14.0,0.0,1,0,0,0,1,0,1


In [67]:
## for customer policy table
customer_policy_details=pd.get_dummies(cpt)

In [68]:
customer_policy_details

,Customer_Id,Annual Premium(Rs),Sales Channel Code,Vintage,Response
0,1.0,40454.0,26.0,217.0,1.0
1,2.0,33536.0,26.0,183.0,0.0
2,3.0,38294.0,26.0,27.0,1.0
3,4.0,28619.0,152.0,203.0,0.0
4,5.0,27496.0,152.0,39.0,0.0
...,...,...,...,...,...
381104,381105.0,30170.0,26.0,88.0,0.0
381105,381106.0,40016.0,152.0,131.0,0.0
381106,381107.0,35118.0,160.0,161.0,0.0
381107,381108.0,44617.0,124.0,74.0,0.0


#### vi. Drop Duplicates (duplicated rows)

In [70]:
## customer details table
customer_details.drop_duplicates()

,Customer_Id,Age,Driving Licence Present,Region Code,Previously Insured,Gender_Female,Gender_Male,Vehicle Age_1-2 Year,Vehicle Age_< 1 Year,Vehicle Age_> 2 Years,Vehicle Damage_No,Vehicle Damage_Yes
0,1.0,44.0,1.0,28.0,0.0,0,1,0,0,1,0,1
1,2.0,76.0,1.0,3.0,0.0,0,1,1,0,0,1,0
2,3.0,47.0,1.0,28.0,0.0,0,1,0,0,1,0,1
3,4.0,21.0,1.0,11.0,1.0,0,1,0,1,0,1,0
4,5.0,29.0,1.0,41.0,1.0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
381104,381105.0,74.0,1.0,26.0,1.0,0,1,1,0,0,1,0
381105,381106.0,30.0,1.0,37.0,1.0,0,1,0,1,0,1,0
381106,381107.0,21.0,1.0,30.0,1.0,0,1,0,1,0,1,0
381107,381108.0,68.0,1.0,14.0,0.0,1,0,0,0,1,0,1


In [71]:
## for customer policy table
customer_policy_details.drop_duplicates()

,Customer_Id,Annual Premium(Rs),Sales Channel Code,Vintage,Response
0,1.0,40454.0,26.0,217.0,1.0
1,2.0,33536.0,26.0,183.0,0.0
2,3.0,38294.0,26.0,27.0,1.0
3,4.0,28619.0,152.0,203.0,0.0
4,5.0,27496.0,152.0,39.0,0.0
...,...,...,...,...,...
381104,381105.0,30170.0,26.0,88.0,0.0
381105,381106.0,40016.0,152.0,131.0,0.0
381106,381107.0,35118.0,160.0,161.0,0.0
381107,381108.0,44617.0,124.0,74.0,0.0


### 3. Create a Master table for future use. Join the customer table and customer_policy table to get a master table using customer_id in both tables.

In [77]:
## join the customer table (cdt) and customer_policy table (cpt) to get a master table using customer_id in both tables.
master_table=pd.merge(cdt,cpt,on='Customer_Id')

In [78]:
master_table['Age']=master_table['Age'].astype('int')
master_table

,Customer_Id,Gender,Age,Driving Licence Present,Region Code,Previously Insured,Vehicle Age,Vehicle Damage,Annual Premium(Rs),Sales Channel Code,Vintage,Response
0,1.0,Male,44,1.0,28.0,0.0,> 2 Years,Yes,40454.0,26.0,217.0,1.0
1,2.0,Male,76,1.0,3.0,0.0,1-2 Year,No,33536.0,26.0,183.0,0.0
2,3.0,Male,47,1.0,28.0,0.0,> 2 Years,Yes,38294.0,26.0,27.0,1.0
3,4.0,Male,21,1.0,11.0,1.0,< 1 Year,No,28619.0,152.0,203.0,0.0
4,5.0,Female,29,1.0,41.0,1.0,< 1 Year,No,27496.0,152.0,39.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
380331,381105.0,Male,74,1.0,26.0,1.0,1-2 Year,No,30170.0,26.0,88.0,0.0
380332,381106.0,Male,30,1.0,37.0,1.0,< 1 Year,No,40016.0,152.0,131.0,0.0
380333,381107.0,Male,21,1.0,30.0,1.0,< 1 Year,No,35118.0,160.0,161.0,0.0
380334,381108.0,Female,68,1.0,14.0,0.0,> 2 Years,Yes,44617.0,124.0,74.0,0.0


### 4. Company needs some important information from the master table to make decisions for future growth.They needs following information:

####  i. Gender wise average annual premium

In [79]:
master_table.groupby(['Gender'])['Annual Premium(Rs)'].mean()

Gender
Female    27409.177761
Male      27279.449195
Name: Annual Premium(Rs), dtype: float64

#### ii. Age wise average annual premium

In [80]:
master_table.groupby(['Age'])['Annual Premium(Rs)'].mean()

Age
20    25249.661665
21    28377.221283
22    28517.344576
23    28386.704147
24    28566.551323
          ...     
81    26380.696388
82    29838.793033
83    26231.409050
84    27571.999918
85    24775.545413
Name: Annual Premium(Rs), Length: 66, dtype: float64

#### iii. Is your data balanced between the genders?

In [81]:
master_table['Gender'].value_counts()

Male      205851
Female    174485
Name: Gender, dtype: int64

#### iv. Vehicle age wise average annual premium.

In [82]:
master_table.groupby(['Vehicle Age'])['Annual Premium(Rs)'].mean()

Vehicle Age
1-2 Year     26840.957357
< 1 Year     27790.008349
> 2 Years    28939.700152
Name: Annual Premium(Rs), dtype: float64

### 5. Is there any relation between Person Age and annual premium?

    Hint: use correlation function (Correlation describes the relationship between two variables). 

    Correlation coefficient < -0.5           - Strong negative relationship

    Correlation coefficient > 0.5            -  Strong positive relationship

    0.5 < Correlation coefficient < 0.5   - There is no relationship. 

In [85]:
correlation = master_table['Age'].corr(master_table['Annual Premium(Rs)'])

In [87]:
if correlation < -0.5:
    print("Strong negative relationship")
elif correlation > 0.5:
    print("Strong positive relationship")
else:
    print("There is no relationship")

There is no relationship
